#Euros SQL

Removing files from dbfs

In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/teaminfo", True)

Out[51]: True

In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/thirdplacementcombinations", True)

Out[52]: True

In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/qualificationplacement", True)


Out[53]: True

Dropping Tables and Views

In [0]:
%sql
DROP TABLE IF EXISTS TeamInfo;
DROP TABLE IF EXISTS GroupStageMatches;
DROP TABLE IF EXISTS GroupStageResults;
DROP TABLE IF EXISTS ThirdPlacementCombinations;
DROP TABLE IF EXISTS QualificationPlacement;

Table For Team Names and Groups

In [0]:
%sql
CREATE OR REPLACE TABLE TeamInfo(
    TeamID INT,
    GroupName CHAR(1),
    TeamName VARCHAR(53)
);

Values for Above Created Table

In [0]:
%sql
INSERT INTO TeamInfo(TeamID, GroupName, TeamName)
VALUES
    (1, 'A', 'Germany'),
    (2, 'A', 'Scotland'),
    (3, 'A', 'Hungary'),
    (4, 'A', 'Switzerland'),
    (5, 'B', 'Spain'),
    (6, 'B', 'Croatia'),
    (7, 'B', 'Italy'),
    (8, 'B', 'Albania'),
    (9, 'C', 'Slovenia'),
    (10, 'C', 'Denmark'),
    (11, 'C', 'Serbia'),
    (12, 'C', 'England'),
    (13, 'D','Netherlands'),
    (14, 'D', 'Austria'),
    (15, 'D', 'France'),
    (16, 'D', 'Poland'),
    (17, 'E', 'Belgium'),
    (18, 'E', 'Slovakia'),
    (19, 'E', 'Romania'),
    (20, 'E', 'Ukraine'),
    (21, 'F', 'Turkey'),
    (22, 'F', 'Portugal'),
    (23, 'F', 'Czech Republic'),
    (24, 'F', 'Greece');

num_affected_rows,num_inserted_rows
24,24


Checking Data Insert

In [0]:
%sql
SELECT * FROM TeamInfo;

TeamID,GroupName,TeamName
1,A,Germany
2,A,Scotland
3,A,Hungary
4,A,Switzerland
5,B,Spain
6,B,Croatia
7,B,Italy
8,B,Albania
9,C,Slovenia
10,C,Denmark


Query for Generating GroupStage Matches

In [0]:
%sql
CREATE OR REPLACE TABLE GroupStageMatches AS
SELECT
    Row_Number() over(order by 1) as MatchID,
    t1.GroupName AS Team1Group,
    t1.TeamID AS Team1ID,
    t1.TeamName AS Team1Name,
    t2.GroupName AS Team2Group,
    t2.TeamID AS Team2ID,
    t2.TeamName AS Team2Name,
    ROUND(RAND() * 6) AS Team1Goals,  -- Random goal value for Team1 with maximun goals 6
    ROUND(RAND() * 6) AS Team2Goals,   -- Random goal value for Team2 with maximun goals 6
    CASE
        WHEN Team1Goals > Team2Goals THEN t1.TeamName
        WHEN Team2Goals > Team1Goals THEN t2.TeamName
        ELSE "Draw"
    END AS MatchWinner
FROM
    TeamInfo t1
CROSS JOIN
    TeamInfo t2
WHERE
    t1.GroupName = t2.GroupName
    AND t1.TeamID < t2.TeamID;  -- TO avoid duplicate matches and self-matches

num_affected_rows,num_inserted_rows


Cheking those Matches

In [0]:
%sql
SELECT MatchID, Team1Name, Team1Goals, Team2Goals, Team2Name, MatchWinner FROM GroupStageMatches;

MatchID,Team1Name,Team1Goals,Team2Goals,Team2Name,MatchWinner
1,Germany,0.0,5.0,Switzerland,Switzerland
2,Germany,4.0,3.0,Hungary,Germany
3,Germany,3.0,2.0,Scotland,Germany
4,Scotland,5.0,1.0,Switzerland,Scotland
5,Scotland,0.0,3.0,Hungary,Hungary
6,Hungary,2.0,5.0,Switzerland,Switzerland
7,Spain,2.0,4.0,Albania,Albania
8,Spain,1.0,1.0,Italy,Draw
9,Spain,2.0,5.0,Croatia,Croatia
10,Croatia,3.0,0.0,Albania,Croatia


Result Table Calculation Accroding to the Above Matches

In [0]:
%sql
CREATE OR REPLACE TABLE GroupStageResults AS
SELECT
    GroupName,
    TeamName,
    COUNT(*) AS MatchesPlayed,
    SUM(CASE WHEN MatchWinner = TeamName THEN 1 ELSE 0 END) AS MatchesWon,
    SUM(CASE WHEN MatchWinner != TeamName AND MatchWinner != 'Draw' THEN 1 ELSE 0 END) AS MatchesLost,
    SUM(CASE WHEN MatchWinner = 'Draw' THEN 1 ELSE 0 END) AS MatchesDrawn,
    SUM(GoalsFor) AS GF,
    SUM(GoalsAgainst) AS GA,
    SUM(GoalsFor - GoalsAgainst) AS GD,
    SUM(CASE WHEN MatchWinner = TeamName THEN 3 WHEN MatchWinner = 'Draw' THEN 1 ELSE 0 END) AS TotalPoints,
    concat(GroupName, ROW_NUMBER() OVER (PARTITION BY GroupName ORDER BY SUM(CASE WHEN MatchWinner = TeamName THEN 3 WHEN MatchWinner = 'Draw' THEN 1 ELSE 0 END) DESC, SUM(GoalsFor - GoalsAgainst) DESC)) AS Position
FROM (
    SELECT
        Team1ID AS TeamID,
        Team1Name AS TeamName,
        Team1Group AS GroupName,
        MatchWinner,
        Team1Goals AS GoalsFor,
        Team2Goals AS GoalsAgainst
    FROM
        GroupStageMatches

    UNION ALL

    SELECT
        Team2ID AS TeamID,
        Team2Name AS TeamName,
        Team2Group AS GroupName,
        MatchWinner,
        Team2Goals AS GoalsFor,
        Team1Goals AS GoalsAgainst
    FROM
        GroupStageMatches
) AS AllTeams
GROUP BY
    TeamID, TeamName, GroupName
ORDER BY
    GroupName, TotalPoints DESC, GD DESC, GF DESC, GA ASC, TeamName;


num_affected_rows,num_inserted_rows


Checking Group Stage Table

In [0]:
%sql
SELECT * FROM GroupStageResults;

GroupName,TeamName,MatchesPlayed,MatchesWon,MatchesLost,MatchesDrawn,GF,GA,GD,TotalPoints,Position
A,Switzerland,3,2,1,0,11.0,7.0,4.0,6,A1
A,Germany,3,2,1,0,7.0,10.0,-3.0,6,A2
A,Scotland,3,1,2,0,7.0,7.0,0.0,3,A3
A,Hungary,3,1,2,0,8.0,9.0,-1.0,3,A4
B,Croatia,3,2,0,1,9.0,3.0,6.0,7,B1
B,Italy,3,1,0,2,4.0,3.0,1.0,5,B2
B,Albania,3,1,2,0,5.0,7.0,-2.0,3,B3
B,Spain,3,0,2,1,5.0,10.0,-5.0,1,B4
C,Denmark,3,3,0,0,15.0,5.0,10.0,9,C1
C,Slovenia,3,2,1,0,12.0,8.0,4.0,6,C2


Checking for all 3rd Position Teams

In [0]:
%sql
SELECT * FROM GroupStageResults WHERE Position LIKE '%3' ORDER BY TotalPoints DESC, GD DESC, GF DESC, GA ASC, TeamName;

GroupName,TeamName,MatchesPlayed,MatchesWon,MatchesLost,MatchesDrawn,GF,GA,GD,TotalPoints,Position
D,France,3,1,1,1,11.0,12.0,-1.0,4,D3
F,Portugal,3,1,1,1,8.0,9.0,-1.0,4,F3
E,Belgium,3,1,2,0,10.0,7.0,3.0,3,E3
A,Scotland,3,1,2,0,7.0,7.0,0.0,3,A3
B,Albania,3,1,2,0,5.0,7.0,-2.0,3,B3
C,England,3,1,2,0,8.0,12.0,-4.0,3,C3


Creating Table for Seeding Possibilities According to 3rd Place Qualification

In [0]:
%sql
CREATE or replace TABLE ThirdPlacementCombinations(
  SId INT,
  QualificationSequence VARCHAR(8),
  B1 CHAR(2),
  C1 CHAR(2),
  E1 CHAR(2),
  F1 CHAR(2)
);

INSERT INTO  ThirdPlacementCombinations (SId, QualificationSequence, B1, C1, E1, F1)
VALUES
    (1, 'A3B3C3D3', 'A3', 'D3', 'B3', 'C3'),
    (2, 'A3B3C3E3', 'A3', 'E3', 'B3', 'C3'),
    (3, 'A3B3C3F3', 'A3', 'F3', 'B3', 'C3'),
    (4, 'A3B3D3E3', 'D3', 'E3', 'A3', 'B3'),
    (5, 'A3B3D3F3', 'D3', 'F3', 'A3', 'B3'),
    (6, 'A3B3E3F3', 'E3', 'F3', 'B3', 'A3'),
    (7, 'A3C3D3E3', 'E3', 'D3', 'C3', 'A3'),
    (8, 'A3C3D3F3', 'F3', 'D3', 'C3', 'A3'),
    (9, 'A3C3E3F3', 'E3', 'F3', 'C3', 'A3'),
    (10, 'A3D3E3F3', 'E3', 'F3', 'D3', 'A3'),
    (11, 'B3C3D3E3', 'E3', 'D3', 'B3', 'C3'),
    (12, 'B3C3D3F3', 'F3', 'D3', 'C3', 'B3'),
    (13, 'B3C3E3F3', 'F3', 'E3', 'C3', 'B3'),
    (14, 'B3D3E3F3', 'F3', 'E3', 'D3', 'B3'),
    (15, 'C3D3E3F3', 'F3', 'E3', 'D3', 'C3');

SELECT * FROM ThirdPlacementCombinations;

SId,QualificationSequence,B1,C1,E1,F1
1,A3B3C3D3,A3,D3,B3,C3
2,A3B3C3E3,A3,E3,B3,C3
3,A3B3C3F3,A3,F3,B3,C3
4,A3B3D3E3,D3,E3,A3,B3
5,A3B3D3F3,D3,F3,A3,B3
6,A3B3E3F3,E3,F3,B3,A3
7,A3C3D3E3,E3,D3,C3,A3
8,A3C3D3F3,F3,D3,C3,A3
9,A3C3E3F3,E3,F3,C3,A3
10,A3D3E3F3,E3,F3,D3,A3


Table for Rounf of 16 Qualification Placement and Matches According to Above Conditions

In [0]:
%sql
CREATE OR REPLACE TABLE QualificationPlacement (
    MatchID INT,
    Team1 VARCHAR(53),
    Team1Goals INT,
    Team2Goals INT,
    Team2 VARCHAR(53),
    MatchWinner VARCHAR(53),
    MatchStage VARCHAR(3)
);

Isolating Placement Combination for this Scenario

In [0]:
%sql
WITH RoundOfSixteenQualified AS
(
    (
        SELECT Position, TeamName
        FROM GroupStageResults
        WHERE Position LIKE '%1' OR Position LIKE '%2'
    )
    UNION ALL
    (
        SELECT Position, TeamName
        FROM GroupStageResults
        WHERE Position LIKE '%3'
        ORDER BY TotalPoints DESC, GD DESC, GF DESC, GA ASC, TeamName
        LIMIT 4
    )
    ORDER BY Position
),
QualificationCompare AS (
  SELECT c.SId, c.B1, c.C1, c.E1, c.F1, pc.PlacementCombination 
  FROM ThirdPlacementCombinations c
  JOIN (
    SELECT REGEXP_REPLACE(CONCAT_WS('', COLLECT_LIST(SUBSTRING(Position,0))), '\\s', '') AS PlacementCombination
    FROM RoundOfSixteenQualified 
    WHERE Position LIKE '%3'
  ) pc
  ON c.QualificationSequence = pc.PlacementCombination
)
  SELECT * FROM QualificationCompare;

SId,B1,C1,E1,F1,PlacementCombination
10,E3,F3,D3,A3,A3D3E3F3


Calculating and Inserting Qualification Placements, Matches

In [0]:
%sql
WITH RoundOfSixteenQualified AS
(
    (
        SELECT Position, TeamName
        FROM GroupStageResults
        WHERE Position LIKE '%1' OR Position LIKE '%2'
    )
    UNION ALL
    (
        SELECT Position, TeamName
        FROM GroupStageResults
        WHERE Position LIKE '%3'
        ORDER BY TotalPoints DESC, GD DESC, GF DESC, GA ASC, TeamName
        LIMIT 4
    )
    ORDER BY Position
),
QualificationCompare AS (
  SELECT c.SId, c.B1, c.C1, c.E1, c.F1, pc.PlacementCombination 
  FROM ThirdPlacementCombinations c
  JOIN (
    SELECT REGEXP_REPLACE(CONCAT_WS('', COLLECT_LIST(SUBSTRING(Position,0))), '\\s', '') AS PlacementCombination
    FROM RoundOfSixteenQualified 
    WHERE Position LIKE '%3'
  ) pc
  ON c.QualificationSequence = pc.PlacementCombination
)
INSERT INTO QualificationPlacement (MatchID, Team1, Team1Goals, Team2Goals, Team2, MatchWinner, MatchStage)
SELECT
    ROW_NUMBER() OVER (ORDER BY 
      CASE t1.Position
        WHEN 'B1' THEN 1
        WHEN 'A1' THEN 2
        WHEN 'F1' THEN 3
        WHEN 'D2' THEN 4
        WHEN 'E1' THEN 5
        WHEN 'D1' THEN 6
        WHEN 'C1' THEN 7
        WHEN 'A2' THEN 8
      END
    ) AS MatchID,
    t1.TeamName AS Team1,
    ROUND(RAND() * 6) AS Team1Goals,
    ROUND(RAND() * 6) AS Team2Goals,
    t2.TeamName AS Team2,
    CASE
        WHEN Team1Goals > Team2Goals THEN t1.TeamName
        WHEN Team2Goals > Team1Goals THEN t2.TeamName
        ELSE
          CASE
            WHEN Team1 < Team2 THEN Team1
            ELSE Team2
          END
    END AS MatchWinner,
    'R16' AS MatchStage

FROM RoundOfSixteenQualified t1
JOIN RoundOfSixteenQualified t2
ON (
    (t1.Position = 'B1' AND t2.Position = (SELECT B1 FROM QualificationCompare)) OR
    (t1.Position = 'A1' AND t2.Position = 'C2') OR
    (t1.Position = 'F1' AND t2.Position = (SELECT F1 FROM QualificationCompare)) OR
    (t1.Position = 'D2' AND t2.Position = 'E2') OR
    (t1.Position = 'E1' AND t2.Position = (SELECT E1 FROM QualificationCompare)) OR
    (t1.Position = 'D1' AND t2.Position = 'F2') OR
    (t1.Position = 'C1' AND t2.Position = (SELECT C1 FROM QualificationCompare)) OR
    (t1.Position = 'A2' AND t2.Position = 'B2')
);


num_affected_rows,num_inserted_rows
8,8


Checking Round Of 16 Results

In [0]:
%sql
SELECT * FROM QualificationPlacement;

MatchID,Team1,Team1Goals,Team2Goals,Team2,MatchWinner,MatchStage
1,Croatia,5,1,Belgium,Croatia,R16
2,Switzerland,0,1,Slovenia,Slovenia,R16
3,Czech Republic,1,2,Scotland,Scotland,R16
4,Austria,5,2,Romania,Austria,R16
5,Slovakia,2,1,France,Slovakia,R16
6,Poland,5,2,Turkey,Poland,R16
7,Denmark,1,5,Portugal,Portugal,R16
8,Germany,2,1,Italy,Germany,R16


Quarter Final

In [0]:
%sql
INSERT INTO QualificationPlacement (MatchID, Team1, Team1Goals, Team2Goals, Team2, MatchWinner, MatchStage)
SELECT
    ROW_NUMBER() OVER (ORDER BY 1) + 8 AS MatchID,
    t1.MatchWinner AS Team1,
    ROUND(RAND() * 6) AS Team1Goals,
    ROUND(RAND() * 6) AS Team2Goals,
    t2.MatchWinner AS Team2,
    'TBD' AS MatchWinner,
    'QF' AS MatchStage

FROM QualificationPlacement t1
CROSS JOIN QualificationPlacement t2
ON
  (t1.MatchID = 1 AND t2.MatchId = 2) OR
  (t1.MatchID = 3 AND t2.MatchId = 4) OR
  (t1.MatchID = 5 AND t2.MatchId = 6) OR
  (t1.MatchID = 7 AND t2.MatchId = 8);

UPDATE QualificationPlacement
SET MatchWinner = 
    CASE
        WHEN Team1Goals > Team2Goals THEN Team1
        WHEN Team1Goals < Team2Goals THEN Team2
        ELSE
            CASE
                WHEN Team1 < Team2 THEN Team1
                ELSE Team2
            END
    END
WHERE MatchStage = 'QF';

num_affected_rows
4


Checking Quarter Final Result

In [0]:
%sql
SELECT * FROM QualificationPlacement WHERE MatchStage = 'QF';

MatchID,Team1,Team1Goals,Team2Goals,Team2,MatchWinner,MatchStage
9,Croatia,1,0,Slovenia,Croatia,QF
10,Scotland,0,5,Austria,Austria,QF
11,Slovakia,2,0,Poland,Slovakia,QF
12,Portugal,5,1,Germany,Portugal,QF


Semi Final

In [0]:
%sql
INSERT INTO QualificationPlacement (MatchID, Team1, Team1Goals, Team2Goals, Team2, MatchWinner, MatchStage)
SELECT
    ROW_NUMBER() OVER (ORDER BY 1) + 12 AS MatchID,
    t1.MatchWinner AS Team1,
    ROUND(RAND() * 6) AS Team1Goals,
    ROUND(RAND() * 6) AS Team2Goals,
    t2.MatchWinner AS Team2,
    'TBD' AS MatchWinner,
    'SF' AS MatchStage

FROM QualificationPlacement t1
CROSS JOIN QualificationPlacement t2
ON
  (t1.MatchID = 9 AND t2.MatchId = 10) OR
  (t1.MatchID = 11 AND t2.MatchId = 12);

UPDATE QualificationPlacement
SET MatchWinner = 
    CASE
        WHEN Team1Goals > Team2Goals THEN Team1
        WHEN Team1Goals < Team2Goals THEN Team2
        ELSE
            CASE
                WHEN Team1 < Team2 THEN Team1
                ELSE Team2
            END
    END
WHERE MatchStage = 'SF';

num_affected_rows
2


Checking Semi Final

In [0]:
%sql
SELECT * FROM QualificationPlacement WHERE MatchStage = 'SF';

MatchID,Team1,Team1Goals,Team2Goals,Team2,MatchWinner,MatchStage
13,Croatia,6,4,Austria,Croatia,SF
14,Slovakia,4,1,Portugal,Slovakia,SF


Third Place

In [0]:
%sql
INSERT INTO QualificationPlacement (MatchID, Team1, Team1Goals, Team2Goals, Team2, MatchWinner, MatchStage)
SELECT
    ROW_NUMBER() OVER (ORDER BY 1) + 14 AS MatchID,
    CASE
      WHEN
        t1.MatchWinner!= t1.Team1 THEN t1.Team1 ELSE t1.Team2
        END AS Team1,
    ROUND(RAND() * 6) AS Team1Goals,
    ROUND(RAND() * 6) AS Team2Goals,
    CASE
      WHEN
        t2.MatchWinner!= t2.Team2 THEN t2.Team2 ELSE t2.Team1
        END AS Team2,
    'TBD' AS MatchWinner,
    'TP' AS MatchStage
FROM QualificationPlacement t1
CROSS JOIN QualificationPlacement t2
ON
  (t1.MatchID = 13 AND t2.MatchId = 14);

UPDATE QualificationPlacement
SET MatchWinner = 
    CASE
        WHEN Team1Goals > Team2Goals THEN Team1
        WHEN Team1Goals < Team2Goals THEN Team2
        ELSE
            CASE
                WHEN Team1 < Team2 THEN Team1
                ELSE Team2
            END
    END
WHERE MatchStage = 'TP';


num_affected_rows
1


Checking Third Place

In [0]:
%sql
SELECT * FROM QualificationPlacement WHERE MatchStage = 'TP';

MatchID,Team1,Team1Goals,Team2Goals,Team2,MatchWinner,MatchStage
15,Austria,5,6,Portugal,Portugal,TP


Final

In [0]:
%sql
INSERT INTO QualificationPlacement (MatchID, Team1, Team1Goals, Team2Goals, Team2, MatchWinner, MatchStage)
SELECT
    ROW_NUMBER() OVER (ORDER BY 1) + 15 AS MatchID,
    t1.MatchWinner AS Team1,
    ROUND(RAND() * 6) AS Team1Goals,
    ROUND(RAND() * 6) AS Team2Goals,
    t2.MatchWinner AS Team2,
    'TBD' AS MatchWinner,
    'FIN' AS MatchStage
FROM QualificationPlacement t1
CROSS JOIN QualificationPlacement t2
ON
  (t1.MatchID = 13 AND t2.MatchId = 14);

UPDATE QualificationPlacement
SET MatchWinner = 
    CASE
        WHEN Team1Goals > Team2Goals THEN Team1
        WHEN Team1Goals < Team2Goals THEN Team2
        ELSE
            CASE
                WHEN Team1 < Team2 THEN Team1
                ELSE Team2
            END
    END
WHERE MatchStage = 'FIN';




num_affected_rows
1


Final Result

In [0]:
%sql
SELECT * FROM QualificationPlacement WHERE MatchStage = 'FIN';

MatchID,Team1,Team1Goals,Team2Goals,Team2,MatchWinner,MatchStage
16,Croatia,2,5,Slovakia,Slovakia,FIN
